In [ ]:
import cv2 as cv
import os
import numpy as np

## Operatii cu video-uri

In [ ]:
def display_video(video_path, window_name='Video', frame_delay = 25):
    cap = cv.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Cannot open video.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video.")
            break
        cv.imshow(window_name, frame)
        if cv.waitKey(frame_delay) & 0xFF == ord('q'):
            break

    cap.release()
    cv.destroyAllWindows()
video_path = "highway.mp4"
display_video(video_path)


In [ ]:
def extract_frames(video_path, output_folder):
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Cannot open video.")
        return

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video.")
            break
        
        frame_filename = os.path.join(output_folder, f"frame_{frame_count:04d}.jpg")
        cv.imwrite(frame_filename, frame)
        frame_count += 1

    cap.release()

video_path = "highway.mp4"
output_folder = "frames"
extract_frames(video_path, output_folder)


### Extragerea foreground-ului folosind diferenta dintre frame-uri

In [ ]:
def foreground_extraction_frame_difference(image_folder, output_video, output_video2, fps= 25):

    images = sorted([img for img in os.listdir(image_folder) if img.endswith(('.jpg'))])

    first_frame = cv.imread(os.path.join(image_folder, images[0]))

    height, width, _ = first_frame.shape
    video = cv.VideoWriter(output_video, cv.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    video2 = cv.VideoWriter(output_video2, cv.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    

    video.release()
    video2.release()

image_folder = 'frames'
output_video = ''
output_video2 = ''
foreground_extraction_frame_difference(image_folder, output_video, output_video2, fps=25)


### Extragerea unei imagini de background

In [ ]:
def background_extraction(image_folder, method, k=None, x=None, y=None):
    
    images = sorted([img for img in os.listdir(image_folder) if img.endswith('.jpg')])
    
    selected_frames = []
    if method == 'mean_first_k':
        selected_frames = [cv.imread(os.path.join(image_folder, images[i])) for i in range(k)]
    
    elif method == 'median_first_k':
        selected_frames = [cv.imread(os.path.join(image_folder, images[i])) for i in range(k)]
    
    elif method == 'mean_range':
        selected_frames = [cv.imread(os.path.join(image_folder, images[i])) for i in range(x, y + 1)]
    
    elif method == 'median_range':
        selected_frames = [cv.imread(os.path.join(image_folder, images[i])) for i in range(x, y + 1)]
    
    else:
        raise ValueError("Invalid method")
    
    
    if method.startswith('mean'):
        background = np.mean(selected_frames, axis=0).astype(np.uint8)
    elif method.startswith('median'):
        background = np.median(selected_frames, axis=0).astype(np.uint8)
    
    return background

### Extragerea foreground-ului folosind background fix

In [ ]:
def foreground_extraction_fixed_background(image_folder, method, output_video, output_video2, k=30,x=None,y=None,fps= 25,processing=False):

    images = sorted([img for img in os.listdir(image_folder) if img.endswith('.jpg')])

    first_frame = cv.imread(os.path.join(image_folder, images[0]))
    height, width, _ = first_frame.shape

    video = cv.VideoWriter(output_video, cv.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    video2 = cv.VideoWriter(output_video2, cv.VideoWriter_fourcc(*'mp4v'), fps, (width, height))


    background = background_extraction(image_folder, method, k, x, y)


    video.release()
    video2.release()

image_folder = 'frames'
output_video = ''
output_video2 = ''
foreground_extraction_fixed_background(image_folder,'median_range', output_video, output_video2,x=470,y=480, fps=25)


### Extragerea foreground-ului folosind background adaptiv

In [ ]:
def foreground_extraction_adaptive_background(image_folder, method, output_video, output_video2, k=10,fps= 25,processing=False):

    images = sorted([img for img in os.listdir(image_folder) if img.endswith('.jpg')])

    first_frame = cv.imread(os.path.join(image_folder, images[0]))
    height, width, _ = first_frame.shape

    video = cv.VideoWriter(output_video, cv.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    video2 = cv.VideoWriter(output_video2, cv.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    
    
    video.release()
    video2.release()

image_folder = 'frames'
output_video = ''
output_video2 = ''
foreground_extraction_adaptive_background(image_folder,'median_range', output_video, output_video2,k=10, fps=25)


### Numararea obiectelor din video

In [ ]:
def calculate_iou(bbox1, bbox2):
    x1, y1, w1, h1 = bbox1
    x2, y2, w2, h2 = bbox2
    x_left = max(x1, x2)
    y_top = max(y1, y2)
    x_right = min(x1 + w1, x2 + w2)
    y_bottom = min(y1 + h1, y2 + h2)

    if x_right <= x_left or y_bottom <= y_top:
        return 0  

    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    area1 = w1 * h1
    area2 = w2 * h2
    union_area = area1 + area2 - intersection_area

    iou = intersection_area / union_area
    return iou

In [ ]:
image_folder = 'frames'
output_video = ''
output_video2 = ''
fps = 25

images = sorted([img for img in os.listdir(image_folder) if img.endswith('.jpg')])

first_frame = cv.imread(os.path.join(image_folder, images[0]))
height, width, _ = first_frame.shape

video = cv.VideoWriter(output_video, cv.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
video2 = cv.VideoWriter(output_video2, cv.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

background = background_extraction(image_folder, 'median_range', k=30, x=470, y=480)

previous_contours = []  
total_passed_count = 0  

for image_name in images:
    

    cv.putText(current_frame, f'{total_passed_count}', (10, 30),
               cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA) 

    video2.write(current_frame)

video.release()
video2.release()
